### Sample code - forks count

In [1]:
import pandas as pd
merge = pd.read_csv('merge_bq_repo_name.csv') # merge_bq_repo_name is the repo name I merged 
merge.shape

(16467, 2)

In [2]:
merge.head(10)

,repo_name,company_name
0,000webhost/000webhost-sdk-js,000webhost
1,100dayproject/polymer-web-example,100dayproject
2,18F/18f-cf-release,18F
3,18F/api-program,18F
4,18F/caddy-boshrelease,18F
5,18F/cf-abacus,18F
6,18F/cf-meteor-buildpack,18F
7,18F/cg-cf-release,18F
8,18F/cg-style,18F
9,18F/cg-style-gem,18F


In [3]:
merge_repo = list(merge['repo_name']) # get only repo name from dataset
len(merge_repo)

16467

### Create pickle to save data in every iteration

In [4]:
# -- save picle  
import pickle
filepath = 'branch_time.pkl'

def batch(time_list):
    
    with open(filepath, 'ab') as f:
        pickle.dump(time_list, f)

In [5]:
import requests
import time
import json
#auth = ('ChuxinYang', '2eb6b92b1736e2b631807e01623fb1dbe7179de9') # this can be get from your github account
auth = ('henseltsai', '672faed0b6f2005bb28a0b5750ee265d7d88a685')
branch_time = []

#for j in range(len(merge_repo)):

for j in range(2055,len(merge_repo)): 

    page = 1
    while True: 
        
        # use github API to scrape data
        get = requests.get('https://api.github.com/repos/'+ str(merge_repo[j]) +'/branches?page='+ str(page) +'&per_page=100',
                      auth = auth)
        get_json = json.loads(get.text)
        
        # test if this repo still exist
        if type(get_json) == dict:
            break
            
        else:
            if len(get_json) == 0: # if there is nothing inside, break out of the loop and look for next repo
                break
 
                
            for i in range(len(get_json)):

                repo_tuple = tuple([merge_repo[j]]) # save the repo name first
                repo_tuple += tuple([get_json[i]['name']])
                repo_tuple += tuple([get_json[i]['commit']['sha']])
                get2 = requests.get('https://api.github.com/repos/'+ str(merge_repo[j]) +'/commits/'+str(get_json[i]['commit']['sha']),auth=auth)
                get2_json = json.loads(get2.text)
                repo_tuple += tuple([get2_json["commit"]["author"]["date"]])
                repo_tuple += tuple([get2_json["commit"]["committer"]["date"]])
                batch(repo_tuple) # call batch function to save repo_tuple in a pickle file
                    
            page += 1 
    print(j) # print which repo we are scraping
    time.sleep(10)

0
1
